In [ ]:
!nvidia-smi

# environment

In [ ]:
!pip install gdown --quiet

### code

In [ ]:
%%bash
File_id="1eSGdjEsfusyZkXwsENAeZlCQE7J7dDjB"
File_name="benchmark"

gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"
unzip -q "${File_name}.zip" -d "${File_name}"
# rm -f "${File_name}.zip"

In [ ]:
%cd benchmark/

### package

In [ ]:
!pip install -r requirements.txt --quiet # --q 讓他安靜

### data

In [ ]:
%%bash
File_id="1eTW_yW4PRip3ZXtpaw5jy4f8MRLRo4BW"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

In [ ]:
%%bash
folder="data/train/helas3_ctcf"

mkdir -p $folder
unzip -q "helas3_ctcf.zip" -d $folder

In [ ]:
!ls

In [ ]:
# !rm -r output

### run

In [ ]:
%%bash
#!/bin/bash
name="2024.05.10_cnn_hyper-test-no-bn"

# Define configuration variables
config="conf/base-CNN.yaml"
input_folder="data/train/helas3_ctcf"
out_folder_base="output/${name}"
input_format="onehot"

learning_rates=(0.001)
hidden_sizes=(300)
extractor_hidden_size=(10 50 100 200)
hidden_layer=(2)

bp_range=(1000 2000 3000)

total_tasks=$((
    ${#learning_rates[@]} * ${#hidden_sizes[@]} *
    ${#hidden_layer[@]} * ${#bp_range[@]} * ${#extractor_hidden_size[@]}
))
completed_tasks=1

# Define hyperparameters (using loops)
for lr in ${learning_rates[@]}; do
    for hs in ${hidden_sizes[@]}; do
        for hl in ${hidden_layer[@]}; do
            for bp in ${bp_range[@]}; do
                for ehs in ${extractor_hidden_size[@]}; do
                  # Construct output folder name with experiment details
                  config="conf/base-CNN_${bp}bp.yaml"
                  input_data="${input_folder}/${bp}bp.50ms.${input_format}/data.h5"
                  out_folder="${out_folder_base}/Lr${lr}_CHs${ehs}_Hs${hs}_Hl${hl}_Bp${bp}"
                    
                  echo ">>>>>>>>>>>>>>>>> Task <<<<<<<<<<<<<<<<<"
                  echo "[$((${completed_tasks}))/${total_tasks}]"
                  echo "Config: ${config}"
                  echo "Input data: ${input_data}"
                  echo "Out folder: ${out_folder}"
                  echo "<<<<<<<<<<<<<<<<< Task >>>>>>>>>>>>>>>>>"

                  # Train with the specified hyperparameters
                  python train.py \
                    --config "${config}" \
                    --input "${input_data}" \
                    --output_folder "${out_folder}" \
                    --device gpu \
                    --eval_freq 1 \
                    --pin_memory_train True \
                    --train.learning_rate $lr \
                    --model.extractor_hidden_size $ehs \
                    --model.extractor_output_dim $hs \
                    --model.classifier_input_dim $hs \
                    --model.classifier_hidden_size $hs \
                    --model.classifier_hidden_layer $hl \
                    --model.extractor_bn False

                  # 檢查執行結果
                  if [ $? -ne 0 ]; then
                    echo "Error: Training failed with batch size $batch_size"
                    exit 1
                  fi

                  ((completed_tasks++))
                done
            done
        done
    done
done

# Sort experiment results
# python code/helper_scripts/sort_exp_result.py --folder "$out_folder_base"

In [ ]:
%%bash
name="2024.05.10_cnn_hyper-test-no-bn"
folder="output"

zip -r "${folder}/${name}.zip" "${folder}/${name}"

In [ ]:
# 一定要在這路徑底下才行
import os
os.chdir('/kaggle/working')

from IPython.display import FileLink
FileLink('benchmark/output/2024.05.10_cnn_hyper-test-no-bn.zip')